<a href="https://colab.research.google.com/github/MRM07/Rep-1/blob/master/Spell_Checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The technology around spell-check and autocorrect has almost outgrown the bandwidth of human error/typos and is now used everywhere: emails, word-processors, search-queries, and even while writing codes. 

Here, we are trying to replicate the basic tech around spell-check which is probably just naive but provides a detailed intutition around the concept and is a very important area in Natural Language Processing.

Spelling Errors are of different types, but the most common ones we come across everyday are- 

1. **Non-Word errors**: Typos, which are the very usual form of errors, missing a few keys, or pressing extra keys in the neighbourhood of your keyboard.

2. **Word-Errors**: Unintentionally, we sometimes end up creating a real word, while we meant sth else, ex- three/there

3. **Cognitive errors**: Can be account for phonetic errors, Maybe the person's grammar is not that strong, or somewhere, we used a speech-text service, and our text result came in this form.



For our convenice, we will be using text file containing a million words, compiled using the excerpts from Project Gutenburg, and most frequent words from Wikitinary and British National Corpus


argmax(c ∈ candidates) P(c|w) 

By **Bayes Theorem**, our equation is of the form ->
**argmax**(c ∈ candidates) P(c)* P(w|c) / P(w)

Since, P(w) is constant value, we write our equation as
**argmax**(c ∈ candidates)P(c)* P(w|c)

Our equation has 4 expressions:
  1. Choosing c which has maximum joint probability
  2. Choosing a set of candidates c, to consider
  3. P(c) calculated from the language model, larger the no. of data, better the      corpus is
  4. P(w|c) Probability that w was typed, when the author meant c. 
     ex P(thew|the) >> P(thedey|the)


In [0]:
import re
from collections import Counter


def words(text):
  return re.findall(r'\w+' ,text.lower())

#word_count_dictionary is a key value pair of  word-frequency
WORD_COUNT_DICTIONARY= Counter(words(open("big.txt").read()))

Levenshtein Distance is a measure of distance between 2 strings. Here ,the difference between the correct string and its incorrect counterpart in terms of the no. of transformations that are needed to be made.

We will first work on the candidate model where we will be able to get back all transformation possible on the word

In [0]:
#if only 1 edit is allowed
def edit1(word):
  
  split_vocab= [(word[:i], word[i:]) for i in range(len(word)+1) ]
  letters= "abcdefghijklmnopqrstuvwxyz"
  delete_op= [L + R[1:] for L, R  in split_vocab if R ]
  transpose_op= [L +R[1] +R[0] +R[2:] for L,R in split_vocab if len(R) > 1]
  substitution_op= [L + C + R[1:] for L, R  in split_vocab if R for C in letters  ]
  insert_op = [L + C + R for L, R  in split_vocab for C in letters]
  return set(insert_op + delete_op + transpose_op + substitution_op)

#if  2 edits are allowed
def edit2(word):
  return ( e2 for e1 in edit1(word) for e2 in edit1(e1) )



We have to also filter out our transformed dataset by only including the words which are present in our actual language model. Let's write a function for that

In [0]:
def filtered(words):
  return set(w for w in words if w in WORD_COUNT_DICTIONARY)

Now, let's work on generating possible spell corrections for our incorrectly spelt words. Basically, write a function which combines the above 2 blocks of function and returns us a candidate model

In [0]:
def candidate_model_vocabulary(word):
  return (filtered([word]) or filtered(edit1(word)) or filtered(edit2(word)) or [word] )

Let's write a function which tells us the probability of a word in the language model.


In [0]:
def Probablity_Scores(word, N=sum(WORD_COUNT_DICTIONARY.values())): 
    "Probability of `word`."
    return WORD_COUNT_DICTIONARY[word] / N

Let's create a function which is going to return us the correct spelling of our incorrectly spelt word based on the probability values of the candidate list of probable words

In [0]:
def correction(word):
  "Most probable spelling correction"
  return max(candidate_model_vocabulary(word), key= Probablity_Scores)

We have built our function. Let's test our code

In [9]:
print(correction('speling'))
print(correction('korrectiun'))

spelling
correction
